In [ ]:
import itertools
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import sqlite3

In [ ]:
db_file = "dgamr.db"
con = sqlite3.connect(db_file)
df = pd.read_sql_query("SELECT * from runs", con)

In [ ]:
"""

Grid convergence - comparing local vs global refinement.

For linear elements we aim to preserve the quadratic convergence in the L2-norm. 
This is not possible with standard global refinement but can be achieved with 
aggressive local refinement at the singularity. 

"""

markers = itertools.cycle(["+", "*", "s", "o", "p"])

max_local_refinement = 10

for local_ref in range(max_local_refinement):
    data = df[df.local_refinements == local_ref]
    if not data.empty:
        plt.loglog(np.sqrt(data.num_dofs), data.error_l2, 
                   label=f"local refinements: {local_ref}", marker=next(markers))
    
x = np.linspace(1e1, 1e3)
y = [3 * 1/xx**2 for xx in x]

plt.loglog(x, y, ls="--", label="$\mathcal{O}(h^2)$")

plt.xlabel("$1/h$")
plt.ylabel("$\|x^* - x_h\|_0$")
plt.grid()
plt.legend()
plt.savefig("error_conv_vs_h.png", dpi=300, bbox_inches="tight")

In [ ]:
"""

For quantitative insight, we now plot the error vs. the number of DoFs instead of 1/h on the x-axis.

"""

markers = itertools.cycle(["+", "*", "s", "o", "p"])

max_local_refinement = 10

for local_ref in range(max_local_refinement):
    data = df[df.local_refinements == local_ref]
    if not data.empty:
        plt.loglog(data.num_dofs, data.error_l2, label=f"local refinements: {local_ref}", 
                   marker=next(markers))

plt.xlabel("DoFs")
plt.ylabel("$\|x^* - x_h\|_0$")
plt.grid()
plt.legend()
plt.savefig("error_conv_vs_dofs.png", dpi=300, bbox_inches="tight")

In [ ]:
con.close()